## Imports

In [1]:
# pip install --upgrade scikit-learn

In [1]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

Pandas   1.1.3
Sklearn  0.23.2


## Get the dataset
- **CLOUD = True**: Download dataset from Kaggle. Necesary for cloud enviroments like COLAB. **Specify your [kaggle credentials](https://www.kaggle.com/docs/api)**.
- **CLOUD = False**: Get the dataset from your local machine. **Specify the data path**.

In [2]:
CLOUD = False

if CLOUD:
    import os
    os.environ['KAGGLE_USERNAME'] = "your_kaggle_username"
    os.environ['KAGGLE_KEY']      = "your_kaggle_api_key"  # See https://www.kaggle.com/docs/api
    !pip install --upgrade kaggle
    !kaggle competitions download -c titanic
    DATA_PATH = "./"

else:
    DATA_PATH = "../../datasets/titanic/"

## Load data

In [3]:
df      = pd.read_csv(DATA_PATH + "train.csv", index_col='PassengerId')
df_test = pd.read_csv(DATA_PATH + "test.csv",  index_col='PassengerId')

print("Train DataFrame:", df.shape)
print("Test DataFrame: ", df_test.shape)

Train DataFrame: (891, 11)
Test DataFrame:  (418, 10)


## Check missings

In [4]:
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [5]:
df_test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

# Exercise 1:
Extract the title (Mr, Mrs, ... ) from the "Name" column.

Tips:
- split(',')[1] to get the 2nd part, and remove the surnamename
- split('.')[0] to get the 1str part, and remove the name

In [6]:
# CODE HERE get_Title_from_Name funtion

def get_Title_from_Name(string):
        string_ret = string.split(", ")[1].split(".")[0]
        return string_ret


df["Title"]      = df['Name'].map(get_Title_from_Name)
df_test["Title"] = df_test['Name'].map(get_Title_from_Name)
df["Title"]

PassengerId
1        Mr
2       Mrs
3      Miss
4       Mrs
5        Mr
       ... 
887     Rev
888    Miss
889    Miss
890      Mr
891      Mr
Name: Title, Length: 891, dtype: object

# Exercise 2:
Apply the title_dictionary to get a better information about the title. You have to overwrite the Title variable.

In [7]:
title_dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty",
    "Dona" : "Royalty"
    
}

In [8]:
# CODE HERE

df["Title"] = [title_dictionary.get(item,item) for item in df["Title"]]
# df_test["Title"] = 
df_test['Title'] = df_test.Title.map(title_dictionary)

df["Title"].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Royalty', 'Officer'], dtype=object)

# Exercise BONUS:
Try to extract some information from the feature **Ticket**

# Exercise BONUS:
Try to extract some information from the feature **Cabin**

In [9]:
df['Cabin'] = df['Cabin'].fillna('N')
df['Cabin'] = df['Cabin'].str[:1]
df['Cabin'] = df['Cabin'].replace(['A'],'1').replace(['B'], '2').replace(['C'], '3').replace(['D'], '4').replace(['E'], '5').replace(['F'], '6').replace(['G'], '7').replace(['T'], '8').replace(['N'], '0')
df['Cabin']

PassengerId
1      0
2      3
3      0
4      3
5      0
      ..
887    0
888    2
889    0
890    3
891    0
Name: Cabin, Length: 891, dtype: object

# Preprocessing
For X data:
- We drop Survived because is the target variable
- We drop Name because we have extracted the Title: Mr, Mrs, ...
- We drop Ticket because it has no information -> see df.Ticket.nunique()
- We drop Cabin because it has a lot of missings (77% are missings)

Then, we identify **numerical** variables and **categorical** variables,

In [10]:
x = df.drop(columns=["Survived", 'Name', 'Ticket', 'Cabin']) # X DATA (WILL BE TRAIN+VALID DATA)
y = df["Survived"] # 0 = No, 1 = Yes

x_test = df_test.drop(columns=['Name', 'Ticket', 'Cabin']) # # X_TEST DATA (NEW DATA)

In [11]:
cat_vars  = ['Sex', 'Embarked', 'Title']         # x.select_dtypes(include=[object]).columns.values.tolist()
num_vars  = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age'] # x.select_dtypes(exclude=[object]).columns.values.tolist()

print("\nNumerical features:\n", num_vars)
print("\nCategorical features:\n", cat_vars)


Numerical features:
 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']

Categorical features:
 ['Sex', 'Embarked', 'Title']


# Exercise 3:
Create a **ColumnTransformer for Tree Models**. Remember:
- Categorical: Some SimpleImputer -> Some Encoder
- Numerical: Some SimpleImputer -> NO Encoder

In [28]:
num_preprocessing = pipeline.Pipeline(steps=[  ('imputer', SimpleImputer(strategy='most_frequent')),
                                                ('scalar', StandardScaler())])

cat_preporcessing = pipeline.Pipeline(steps=[  ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
                                                ('scalar', StandardScaler()),
                                                ('onehot', OneHotEncoder(handle_unknown='ignore'))])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_preprocessing, num_vars),
    ('cat', cat_preporcessing, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('scalar', StandardScaler())]),
                                 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('ordinal',
                                                  OrdinalEncoder())]),
                                 ['Sex', 'Embarked', 'Title'])])

In [125]:
num_4_treeModels = pipeline.Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='constant', fill_value=-9999)),
])

cat_4_treeModels = pipeline.Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
    ('ordinal', preprocessing.OrdinalEncoder()) # handle_unknown='ignore' ONLY IN VERSION 0.24
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels, num_vars),
    ('cat', cat_4_treeModels, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value=-9999,
                                                                strategy='constant'))]),
                                 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('ordinal',
                                                  OrdinalEncoder())]),
                                 ['Sex', 'Embarked', 'Title'])])

# Exercise 4
1. Complete the diccionary with some Tree Models.
2. Then we put each model in a Pipeline where:
   - first is the prepocessing with the column Transformer
   - Then is the Tree model
3. Display the fullpipeline of the LGBMClassifier

In [13]:
!pip3 install xgboost
!pip3 install lightgbm
!pip3 install catboost

In [29]:
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier

In [30]:
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees": ExtraTreesClassifier(),
  "Random Forest": RandomForestClassifier(),
  "AdaBoost": AdaBoostClassifier(),
  "Skl GBM": GradientBoostingClassifier(),
  "Skl HistGBM": HistGradientBoostingClassifier(),
  "XGBoost": XGBClassifier(),
  "LightGBM": LGBMClassifier(),
  "CatBoost": CatBoostClassifier()
}

tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}

tree_classifiers['LightGBM']

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('scalar',
                                                                   StandardScaler())]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder())]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('lgbmclassifier', LGBMClassifier())])

# Exercise 5:
Define a simple split validation strategy with:
- 80% for train
- 20% for validation
- With stratification
- random_state=0

And train all the models in a for loop

In [31]:
x_train, x_val, y_train, y_val = model_selection.train_test_split(x, y, test_size=0.20, stratify=None, random_state=0)

x_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
PassengerId,,,,,,,,
141,3,female,NaN,0,2,15.2458,C,Mrs
440,2,male,31.0,0,0,10.5000,S,Mr
818,2,male,31.0,1,1,37.0042,C,Mr
379,3,male,20.0,0,0,4.0125,C,Mr
492,3,male,21.0,0,0,7.2500,S,Mr
...,...,...,...,...,...,...,...,...
836,1,female,39.0,1,1,83.1583,C,Miss
193,3,female,19.0,1,0,7.8542,S,Miss
630,3,male,NaN,0,0,7.7333,Q,Mr


In [32]:
for model_name, model in tree_classifiers.items():
    model.fit(x_train,y_train)
    y_pred = model.predict(x_val)
    # TRAIN PIPELINE (PREPRO + MODEL) WITH TRAIN DATA
    # EVAL PIPELINE WITH VAL DATA (SEE ACCURACY AND BALANCED_ACCURACY)

C:\Users\kovac\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:47:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.008911
0:	learn: 0.6872497	total: 56.9ms	remaining: 56.9s
1:	learn: 0.6812175	total: 58.4ms	remaining: 29.1s
2:	learn: 0.6756573	total: 60.1ms	remaining: 20s
3:	learn: 0.6711977	total: 61ms	remaining: 15.2s
4:	learn: 0.6668320	total: 61.9ms	remaining: 12.3s
5:	learn: 0.6628505	total: 63ms	remaining: 10.4s
6:	learn: 0.6580525	total: 64ms	remaining: 9.08s
7:	learn: 0.6535724	total: 65.2ms	remaining: 8.08s
8:	learn: 0.6486509	total: 66.6ms	remaining: 7.33s
9:	learn: 0.6436128	total: 68.2ms	remaining: 6.75s
10:	learn: 0.6404949	total: 69.8ms	remaining: 6.28s
11:	learn: 0.6365291	total: 71.2ms	remaining: 5.86s
12:	learn: 0.6319231	total: 72.6ms	remaining: 5.51s
13

178:	learn: 0.3979555	total: 313ms	remaining: 1.44s
179:	learn: 0.3974986	total: 315ms	remaining: 1.43s
180:	learn: 0.3972108	total: 317ms	remaining: 1.43s
181:	learn: 0.3969160	total: 318ms	remaining: 1.43s
182:	learn: 0.3966936	total: 319ms	remaining: 1.42s
183:	learn: 0.3961763	total: 320ms	remaining: 1.42s
184:	learn: 0.3956810	total: 322ms	remaining: 1.42s
185:	learn: 0.3954344	total: 323ms	remaining: 1.42s
186:	learn: 0.3953641	total: 324ms	remaining: 1.41s
187:	learn: 0.3948890	total: 326ms	remaining: 1.41s
188:	learn: 0.3944836	total: 327ms	remaining: 1.41s
189:	learn: 0.3942896	total: 329ms	remaining: 1.4s
190:	learn: 0.3941107	total: 330ms	remaining: 1.4s
191:	learn: 0.3937130	total: 332ms	remaining: 1.4s
192:	learn: 0.3933582	total: 334ms	remaining: 1.39s
193:	learn: 0.3931460	total: 336ms	remaining: 1.39s
194:	learn: 0.3926819	total: 337ms	remaining: 1.39s
195:	learn: 0.3924322	total: 339ms	remaining: 1.39s
196:	learn: 0.3920037	total: 341ms	remaining: 1.39s
197:	learn: 0.3

389:	learn: 0.3509198	total: 623ms	remaining: 975ms
390:	learn: 0.3507971	total: 625ms	remaining: 973ms
391:	learn: 0.3507641	total: 626ms	remaining: 970ms
392:	learn: 0.3507423	total: 627ms	remaining: 969ms
393:	learn: 0.3506352	total: 629ms	remaining: 967ms
394:	learn: 0.3504911	total: 631ms	remaining: 966ms
395:	learn: 0.3503490	total: 632ms	remaining: 965ms
396:	learn: 0.3502602	total: 634ms	remaining: 963ms
397:	learn: 0.3499687	total: 636ms	remaining: 961ms
398:	learn: 0.3497975	total: 637ms	remaining: 959ms
399:	learn: 0.3495343	total: 639ms	remaining: 958ms
400:	learn: 0.3494157	total: 640ms	remaining: 956ms
401:	learn: 0.3493945	total: 641ms	remaining: 953ms
402:	learn: 0.3493276	total: 642ms	remaining: 951ms
403:	learn: 0.3492615	total: 644ms	remaining: 949ms
404:	learn: 0.3491458	total: 645ms	remaining: 947ms
405:	learn: 0.3490002	total: 646ms	remaining: 946ms
406:	learn: 0.3487544	total: 648ms	remaining: 944ms
407:	learn: 0.3487040	total: 650ms	remaining: 942ms
408:	learn: 

605:	learn: 0.3232478	total: 937ms	remaining: 609ms
606:	learn: 0.3232028	total: 939ms	remaining: 608ms
607:	learn: 0.3230442	total: 940ms	remaining: 606ms
608:	learn: 0.3229986	total: 942ms	remaining: 605ms
609:	learn: 0.3227901	total: 943ms	remaining: 603ms
610:	learn: 0.3227476	total: 945ms	remaining: 601ms
611:	learn: 0.3226377	total: 946ms	remaining: 600ms
612:	learn: 0.3225364	total: 948ms	remaining: 598ms
613:	learn: 0.3224532	total: 949ms	remaining: 597ms
614:	learn: 0.3223691	total: 951ms	remaining: 595ms
615:	learn: 0.3223069	total: 952ms	remaining: 594ms
616:	learn: 0.3222095	total: 954ms	remaining: 592ms
617:	learn: 0.3220795	total: 955ms	remaining: 591ms
618:	learn: 0.3219145	total: 957ms	remaining: 589ms
619:	learn: 0.3218132	total: 958ms	remaining: 587ms
620:	learn: 0.3217791	total: 960ms	remaining: 586ms
621:	learn: 0.3217336	total: 962ms	remaining: 584ms
622:	learn: 0.3216004	total: 963ms	remaining: 583ms
623:	learn: 0.3215439	total: 964ms	remaining: 581ms
624:	learn: 

818:	learn: 0.2999892	total: 1.25s	remaining: 276ms
819:	learn: 0.2998593	total: 1.25s	remaining: 275ms
820:	learn: 0.2997423	total: 1.25s	remaining: 273ms
821:	learn: 0.2996690	total: 1.25s	remaining: 272ms
822:	learn: 0.2996052	total: 1.25s	remaining: 270ms
823:	learn: 0.2995446	total: 1.26s	remaining: 268ms
824:	learn: 0.2993779	total: 1.26s	remaining: 267ms
825:	learn: 0.2991727	total: 1.26s	remaining: 265ms
826:	learn: 0.2990580	total: 1.26s	remaining: 264ms
827:	learn: 0.2990246	total: 1.26s	remaining: 262ms
828:	learn: 0.2989336	total: 1.26s	remaining: 261ms
829:	learn: 0.2987875	total: 1.27s	remaining: 259ms
830:	learn: 0.2987786	total: 1.27s	remaining: 258ms
831:	learn: 0.2987419	total: 1.27s	remaining: 256ms
832:	learn: 0.2985541	total: 1.27s	remaining: 255ms
833:	learn: 0.2984552	total: 1.27s	remaining: 253ms
834:	learn: 0.2983868	total: 1.27s	remaining: 252ms
835:	learn: 0.2983478	total: 1.27s	remaining: 250ms
836:	learn: 0.2982459	total: 1.28s	remaining: 249ms
837:	learn: 

# Exercise 6:
Define a 10 Fold cross validation strategy with:
- With stratification
- shuffle=True
- random_state=0

And train all the models in a for loop.

Tip you can use **[cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)** for both training and predict with 

In [ ]:
skf = model_selection.StratifiedKFold(
    # CODE HERE
)

for model_name, model in tree_classifiers.items():
    # CODE HERE
    # TRAIN PIPELINE (PREPRO + MODEL) WITH TRAIN DATA
    # EVAL PIPELINE WITH VAL DATA (SEE ACCURACY AND BALANCED_ACCURACY)

In [ ]:
skf = model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

for model_name, model in tree_classifiers.items():    

    for i, train_index, test_index in skf.split(x, y):
        print(i, fold)

        x_train = x[train_index]
        y_train = y[train_index]
        x_valid = x[test_index]
        y_valid = y[test_index]

        model.fit(x_train, y_train)

        OOF = model.predict(x_valid)


# Exercise 7
Train **with all data** the best model

In [13]:
best_model = # Select your best model
# Train with all data your best model

# Exercise 8
With your best model, generate the predicitions for test data (x_test)

In [23]:
test_pred = # Get the predictions for x_test

# Exercise 9

Submit to kaggle using the kaggle API. And send us your score. You can try to improve it.

In [ ]:
sub = pd.DataFrame(test_pred, index=x_test.index, columns=["Survived"])
sub.head()

In [ ]:
sub.to_csv("sub.csv")

In [ ]:
!kaggle competitions submit -c titanic -f sub.csv -m "My submission message"

# Exercise BONUS

Knowing how to export your models is very important for putting models in production. Try to
- Export and Load the ColumTransformer in pickle
- Export and Load the ColumTransformer in joblib
- Export and load the Pipeline